## Importação de Bibliotecas

In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2
import requests
from bs4 import BeautifulSoup
import io
import numpy as np
from PIL import Image
import face_recognition

## Funções Auxiliares

### Função para validar documentos com detecção de rosto

In [2]:
def validate_document(file_content):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = Image.open(io.BytesIO(file_content))
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return len(faces) > 0

### Função para validar links de perfis

In [3]:
def validate_profile_link(link, interests):
    try:
        response = requests.get(link, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text().lower()
        return any(interest.lower() in text for interest in interests.split(','))
    except:
        return False

### Função para comparar a foto do documento com a selfie da pessoa

In [4]:
def compare_selfie_with_document(selfie_content, document_content, threshold=0.6):
    try:
        selfie_img = face_recognition.load_image_file(io.BytesIO(selfie_content))
        document_img = face_recognition.load_image_file(io.BytesIO(document_content))

        selfie_face_locations = face_recognition.face_locations(selfie_img)
        document_face_locations = face_recognition.face_locations(document_img)

        if not selfie_face_locations or not document_face_locations:
            return False, 1.0  # Não encontrou faces em uma ou ambas as imagens

        # Extrai os encodings (características) das faces
        selfie_face_encoding = face_recognition.face_encodings(selfie_img, selfie_face_locations)[0]
        document_face_encoding = face_recognition.face_encodings(document_img, document_face_locations)[0]

        # Calcula a distância entre as faces (menor valor = mais similar)
        face_distance = face_recognition.face_distance([selfie_face_encoding], document_face_encoding)[0]

        # Compara se a distância é menor que o limiar
        match = face_distance <= threshold

        return match, face_distance
    except Exception as e:
        print(f"Erro ao comparar faces: {e}")
        return False, 1.0

## Seções do formulário: 

### Seção 1: Dados Pessoais

In [5]:
name_input = widgets.Text(description="Nome:")
address_input = widgets.Text(description="Endereço:")
cpf_input = widgets.Text(description="CPF:")
display(name_input, address_input, cpf_input)

Text(value='', description='Nome:')

Text(value='', description='Endereço:')

Text(value='', description='CPF:')

### Seção 2: Interesses

In [6]:
interests_input = widgets.Textarea(description="Interesses (ex: jogos, times):", rows=3)
display(interests_input)

Textarea(value='', description='Interesses (ex: jogos, times):', rows=3)

### Seção 3: Upload de Documento

In [13]:
### Seção 3: Upload de Documento

doc_upload = widgets.FileUpload(
    description="Upload de Documento", 
    accept=".jpg,.jpeg,.png,.pdf,.bmp",  # Aceita mais formatos
    multiple=False  # Permite apenas um arquivo
)
doc_output = widgets.Output()
display(doc_upload, doc_output)

# Variável para armazenar o conteúdo do documento
document_content = None

def on_doc_upload_change(change):
    global document_content
    with doc_output:
        clear_output()
        if doc_upload.value:
            try:
                # Tenta extrair o conteúdo do arquivo
                if isinstance(doc_upload.value, tuple) and len(doc_upload.value) > 0:
                    # Se for uma tupla, vamos tentar acessar o conteúdo diretamente do objeto
                    if hasattr(doc_upload.value[0], 'content'):
                        file_content = doc_upload.value[0].content  # Acessa o atributo content
                    elif hasattr(doc_upload.value[0], '_data'):
                        file_content = doc_upload.value[0]._data  # Tenta acessar _data
                    elif hasattr(doc_upload.value[0], 'data'):
                        file_content = doc_upload.value[0].data  # Tenta acessar data
                    elif hasattr(doc_upload.value[0], 'value'):
                        file_content = doc_upload.value[0].value  # Tenta acessar value
                    else:
                        # Último recurso: tentativa de conversão direta
                        print("Tentando conversão direta do objeto para bytes...")
                        file_content = bytes(doc_upload.value[0])
                else:
                    print("Formato de upload não reconhecido.")
                    return
                
                document_content = file_content
                
                # Mostra informações sobre o arquivo
                print(f"Documento carregado! Tamanho: {len(file_content) / 1024:.1f} KB")
                
                if validate_document(file_content):
                    print("Validando documento com IA... Face detectada com sucesso!")
                else:
                    print("Validando documento com IA... Aviso: Nenhum rosto foi detectado.")
                    print("Você ainda pode prosseguir, mas a verificação pode não funcionar corretamente.")
            except Exception as e:
                print(f"Erro ao processar arquivo: {e}")
                print(f"Tipo do objeto: {type(doc_upload.value)}")
                if isinstance(doc_upload.value, tuple) and len(doc_upload.value) > 0:
                    print(f"Tipo do primeiro elemento: {type(doc_upload.value[0])}")
                    print(f"Atributos disponíveis: {dir(doc_upload.value[0])}")
                document_content = None

doc_upload.observe(on_doc_upload_change, names='value')

FileUpload(value=(), accept='.jpg,.jpeg,.png,.pdf,.bmp', description='Upload de Documento')

Output()

### Seção 4: Redes Sociais

In [8]:
social_input = widgets.Text(description="Redes Sociais (ex: @usuario):")
social_output = widgets.Output()
display(social_input, social_output)

def on_social_change(change):
    with social_output:
        clear_output()
        if social_input.value:
            print(f"Simulando leitura de interações... Encontradas 50 interações relacionadas a esports.")
social_input.observe(on_social_change, names='value')

Text(value='', description='Redes Sociais (ex: @usuario):')

Output()

### Seção 5: Links de Perfis

In [9]:
profile_link_input = widgets.Text(description="Link de Perfil:")
profile_output = widgets.Output()
display(profile_link_input, profile_output)

def on_link_change(change):
    with profile_output:
        clear_output()
        if profile_link_input.value and validate_profile_link(profile_link_input.value, interests_input.value):
            print("Validando conteúdo com IA... Conteúdo relevante ao perfil!")
        elif profile_link_input.value:
            print("Validando conteúdo com IA... Conteúdo não relevante ou link inválido.")
profile_link_input.observe(on_link_change, names='value')

Text(value='', description='Link de Perfil:')

Output()

### Seção 6: Verificação de Identidade por Selfie

In [17]:
### Seção 6: Verificação de Identidade por Selfie

selfie_upload = widgets.FileUpload(
    description="Upload de Selfie", 
    accept=".jpg,.jpeg,.png,.bmp",  # Formatos de imagem
    multiple=False  # Permite apenas um arquivo
)
verify_button = widgets.Button(description="Verificar Identidade")
selfie_output = widgets.Output()  # Adicionado para mostrar feedback do upload
verify_output = widgets.Output()

display(selfie_upload, selfie_output, verify_button, verify_output)  # Adicionei selfie_output ao display

# Variável para armazenar o conteúdo da selfie
selfie_content = None

# Manipulador para o upload de selfie
def on_selfie_upload_change(change):
    global selfie_content
    with selfie_output:  # Use a saída específica para o upload
        clear_output()
        if selfie_upload.value:
            try:
                # Tenta extrair o conteúdo do arquivo
                if isinstance(selfie_upload.value, tuple) and len(selfie_upload.value) > 0:
                    # Se for uma tupla, vamos tentar acessar o conteúdo diretamente do objeto
                    if hasattr(selfie_upload.value[0], 'content'):
                        file_content = selfie_upload.value[0].content  # Acessa o atributo content
                    elif hasattr(selfie_upload.value[0], '_data'):
                        file_content = selfie_upload.value[0]._data  # Tenta acessar _data
                    elif hasattr(selfie_upload.value[0], 'data'):
                        file_content = selfie_upload.value[0].data  # Tenta acessar data
                    elif hasattr(selfie_upload.value[0], 'value'):
                        file_content = selfie_upload.value[0].value  # Tenta acessar value
                    else:
                        # Último recurso: tentativa de conversão direta
                        print("Tentando conversão direta do objeto para bytes...")
                        file_content = bytes(selfie_upload.value[0])
                else:
                    print("Formato de upload não reconhecido.")
                    return
                
                selfie_content = file_content
                print(f"Selfie carregada! Tamanho: {len(file_content) / 1024:.1f} KB")
            except Exception as e:
                print(f"Erro ao processar selfie: {e}")
                print(f"Tipo do objeto: {type(selfie_upload.value)}")
                if isinstance(selfie_upload.value, tuple) and len(selfie_upload.value) > 0:
                    print(f"Tipo do primeiro elemento: {type(selfie_upload.value[0])}")
                    print(f"Atributos disponíveis: {dir(selfie_upload.value[0])}")
                selfie_content = None

# Manipulador para o botão de verificação
### Adicione este código após a função on_selfie_upload_change

# Manipulador para o botão de verificação
def on_verify_button_click(b):
    with verify_output:
        clear_output()
        if not document_content:
            print("Erro: Nenhum documento válido foi carregado!")
            return
        
        if not selfie_content:
            print("Erro: Nenhuma selfie foi carregada!")
            return
        
        print("Verificando identidade...")
        print("-" * 50)
        
        # Verificação com threshold padrão
        match, similarity = compare_selfie_with_document(selfie_content, document_content)
        confidence = round((1 - similarity) * 100, 2)
        
        if match:
            print(f"✅ VERIFICAÇÃO CONCLUÍDA: As imagens correspondem à mesma pessoa!")
            print(f"Nível de confiança: {confidence}%")
            print("-" * 50)
            print("USUÁRIO VERIFICADO - Acesso liberado aos recursos exclusivos para fãs FURIA")
        else:
            print(f"❌ VERIFICAÇÃO FALHOU: As imagens não correspondem ou houve um erro.")
            print(f"Nível de confiança: {confidence}%")
            print("-" * 50)
            print("Recomendações:")
            print("- Tente fazer upload de uma selfie com melhor iluminação")
            print("- Certifique-se que seu rosto está claramente visível")
            print("- Verifique se o documento possui uma foto nítida")


# Registra os manipuladores de eventos
selfie_upload.observe(on_selfie_upload_change, names='value')
verify_button.on_click(on_verify_button_click)  # Adicione o evento de clique do botão

FileUpload(value=(), accept='.jpg,.jpeg,.png,.bmp', description='Upload de Selfie')

Output()

Button(description='Verificar Identidade', style=ButtonStyle())

Output()

## Instruções de Uso

1. **Dados Pessoais**: Insira seu nome, endereço e CPF nos campos fornecidos.
2. **Interesses**: Descreva seus interesses em esports, como jogos favoritos, times (ex: FURIA), eventos e compras relacionadas.
3. **Upload de Documento**: Faça upload de uma imagem de um documento de identificação. A IA detectará se há um rosto na imagem.
4. **Redes Sociais**: Insira seu perfil de rede social (ex: Twitter, Instagram). A simulação exibirá interações relacionadas a esports.
5. **Links de Perfis**: Insira um link para seu perfil em um site de esports. A IA verificará se o conteúdo é relevante com base nos seus interesses.